# I Exact simulation of one-dimensional neuronal model

## 1 Wiener process with drift :
$$
dV(t) = Idt + \sigma dB(t),\quad V(0) = V_{0}
$$
Exact distribution of (V(t)) : (cf lecture)
$$
V(t_{i})= V(t_{i-1}) + I\Delta + \mathcal{N}(0,\Delta \sigma^{2})
$$

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import math
import plotly
#from plotly import __version__
py=plotly.offline
go =plotly.graph_objs
plotly.offline.init_notebook_mode()

def histogram(data,title,dataName):
    trace= []
    for i in range(0,len(data)):
        histo = go.Histogram(
            x=data[i],
            #histnorm='probability',
            opacity=0.5
        )
        trace.append(histo)
    layout = go.Layout(
    title=title,
    xaxis=dict(
        title=dataName
    ),
    yaxis=dict(
        title='Count'
    ),
    barmode='stack',
    bargap=0.25,
    bargroupgap=0.3
    )
    fig = go.Figure(data=trace, layout=layout)
    #py.plot(fig, filename='histogram/histogramIsBooking'+data_name+'Histogram.html')
    py.iplot(fig)
    
def scatter(X_name,Y,Y_name,title,X=0,histo=0):
    if X == 0 :
        X = [ x for x in range(0,len(Y[0]))];
    data = []
    for i in range(0,len(Y)):
        data.append(go.Scatter(
        x=X,
        y=Y[i],
        mode = 'lines',
        name = Y_name[i]
    ))
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title=X_name,
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='value',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [2]:
def Wiener(delta, numberOfTrials, I, V0, sigma):
    Vlist = []
    sigmaName = []
    for j in sigma:
        V=[V0];
        for i in range(1,numberOfTrials):
            V.append(V[i-1] + I*delta + np.random.normal(0,(math.sqrt(delta))*j))
        Vlist.append(V)  
        sigmaName.append("sigma = "+str(j))
    scatter("time",Vlist,sigmaName,"Wiener process ")

  #      plt.plot(V,label="Wiener process with sigma = "+str(j))
  #      plt.title("Wiener process")
  #      plt.xlabel("time")
  #      plt.xlabel("Wiener process")
  #      plt.legend(loc='best')
  #  plt.show()
    

In [3]:
Wiener(0.0001,1000,-10,-65,[1,10,20])

## 2 Ornstein-Uhlenbeck process : 
$$
dV(t) = (-\frac{V(t) - \alpha}{\tau})dt + \sigma dB(t),\quad V(0) = V_{0},\quad with\quad \alpha = V_{0} + \tau I
$$
Exact distribution of (V(t)) : (cf lecture)
$$
V(t_{i})= V(t_{i-1})\exp(\frac{-\Delta}{\tau}) +\tau (\frac{V_{0}}{\tau} + I)(1-\exp(\frac{-\Delta}{\tau}) + \mathcal{N}(0,\sigma \sqrt{\frac{\tau}{2}(1-\exp(-2\frac{\Delta}{\tau}})
$$

In [4]:
def OU(delta, numberOfTrials,tau, I, V0, sigma,S):
    V=[V0];
    spikeDistrib = [0]
    spikingTime = 0
    spikes = []
    for i in range(1,numberOfTrials):
        Vi = V[i-1]*math.exp((-delta/tau)) + (V0/tau + I)*tau*(1-math.exp(-(delta)/tau)) + np.random.normal(0,(math.sqrt((tau/2)*(1-math.exp(-2*(delta)/tau))))*sigma)
        if Vi > S :
            V.append(V0);
            spikes.append(spikingTime)
            spikeDistrib.append( spikingTime-spikeDistrib[-1])
        else:
            V.append(Vi);
            spikingTime +=1 ;
            
    scatter("time",[V],['sigma = '+str(sigma)],"Ornstein-Uhlenbeck process ")
    histogram([spikeDistrib],'Spike Distribution','time')
    return V
    #plt.subplot(211)
    #plt.plot(V,label="Ornstein-Uhlenbeck process with sigma = "+str(sigma))
    #plt.title("Ornstein-Uhlenbeck process")
    #plt.xlabel("time")
    #plt.ylabel("Ornstein-Uhlenbeck process")
    #for i in range(#0,len(spikes)):
    #    plt.axvline(spikes[i],color='r')
    #plt.legend()
    #plt.subplot(212)
#
    #plt.hist(spikeDistrib,bins=len(set(spikeDistrib)))
    ##print(spikeDistrib)
    #plt.title("Spike Distribution")
    #plt.xlabel("time")
    #plt.ylabel("occurences")/tau + I
    #plt.show()

In [5]:
V = OU(0.001,5000,0.5,50,-65,10,-45)

In [6]:
V = OU(0.001,5000,0.5,20,-65,10,-45)

# II Approximate simulation of one-dimensional neuronal model

## 1 Ornstein-Uhlenbeck process :

### Euler scheme for the OU process :

Set $ (V_{t})$ a solution of $ dV(t) = (-\frac{V(t) - \alpha}{\tau})dt + \sigma dB(t),\quad V(0) = V_{0},\quad with\quad \alpha = V_{0} + \tau I $.

We want to approximate $(V_{t})$ on [0,T]. We start by descretizing the time interval : $0=t_{0}<t_{1}<...<t_{N}=T$.

Let $\Delta_{j} = t_{j+1} - t_{j} \simeq dt \rightarrow B_{t_{j+1}} - B_{t_{j}} \simeq dB_{t}$. 

The Euler-Maruyama approximation of $(V_{t})$  is $(X_{t})$ defined by : 
\begin{equation}
\begin{split}
 X_{0} &= V_{0}\\
 X_{t_{1}} - X_{t_{0}} &= (-\frac{X(t_{0}) - \alpha}{\tau})(t_{1}-t_{0}) + \sigma (B_{t_{1}} - B_{t_{0}})\\
 &=\quad ... \\
\forall j=1..(N-1):\quad X_{t_{j+1}} &= X_{t_{j}} + (-\frac{X(t_{j}) - \alpha}{\tau})\Delta_{j} + \sigma(B_{t_{j+1}} - B_{t_{j}})\\
\end{split}
\end{equation}
With : $ (B_{t_{j+1}} - B_{t_{j}}) \sim \mathcal{N}(0,\Delta_{j})$


In [7]:
def EulerOU(delta, numberOfTrials,tau, I, V0, sigma,S):
    X=[V0];
    alpha = V0 + tau*I;
    spikeDistrib = [0]
    spikingTime = 0
    spikes = []
    for i in range(1,numberOfTrials):
        Xi = X[i-1] + ((-X[i-1] + alpha)/tau)*delta + sigma*np.random.normal(0,delta)
        if Xi > S :
            X.append(V0);
            spikes.append(spikingTime)
            spikeDistrib.append( spikingTime-spikeDistrib[-1])
        else:
            X.append(Xi);
            spikingTime +=1 ;
            
    scatter("time",[X],['sigma = '+str(sigma)],"Euler scheme : Ornstein-Uhlenbeck process ")    
    histogram([spikeDistrib],'Spike Distribution','time')
    return X

In [8]:
 X = EulerOU(0.001,5000,0.5,50,-65,10,-45)

In [10]:
diff = []
for i in range(1,len(X)):
    diff.append(V[i] - X[i])

In [11]:
scatter("time",[diff],[str(10)],"Euler scheme vs Ornstein-Uhlenbeck process ")    

## 2 Feller process :

### Euler scheme for the OU process :

Set $ (V_{t})$ a solution of $ dV(t) = (-\frac{V(t) - \alpha}{\tau})dt + \sigma\sqrt{V_{t} - V_{I}} dB(t),\quad V(0) = V_{0},\quad with\quad \alpha = V_{0} + \tau I $.

We want to approximate $(V_{t})$ on [0,T]. We start by descretizing the time interval : $0=t_{0}<t_{1}<...<t_{N}=T$.

Let $\Delta_{j} = t_{j+1} - t_{j} \simeq dt \rightarrow B_{t_{j+1}} - B_{t_{j}} \simeq dB_{t}$. 

The Euler-Maruyama approximation of $(V_{t})$  is $(X_{t})$ defined by : 
\begin{equation}
\begin{split}
 X_{0} &= V_{0}\\
 X_{t_{1}} - X_{t_{0}} &= (-\frac{X(t_{0}) - \alpha}{\tau})(t_{1}-t_{0}) + \sigma\sqrt{X_{t_{j}} - V_{I}} (B_{t_{1}} - B_{t_{0}})\\
 &=\quad ... \\
\forall j=1..(N-1):\quad X_{t_{j+1}} &= X_{t_{j}} + (-\frac{X(t_{j}) - \alpha}{\tau})\Delta_{j} + \sigma\sqrt{X_{t_{j}} - V_{I}}(B_{t_{j+1}} - B_{t_{j}})\\
\end{split}
\end{equation}
With : $ (B_{t_{j+1}} - B_{t_{j}}) \sim \mathcal{N}(0,\Delta_{j})$

In [12]:
def EulerFeller(delta, numberOfTrials,tau, I, V0,VI, sigma,S):
    alpha = V0 + tau*I;
    VIname=[]
    Xlist=[]
    spikeDistribList = []
    for j in range(0,len(VI)):
        X=[V0];
        spikeDistrib = [0]
        spikingTime = 0
        spikes = []
        VIname.append("VI = "+str(VI[j]));
        for i in range(1,numberOfTrials):
            Xi = X[i-1] + ((-X[i-1] + alpha)/tau)*delta + sigma*math.sqrt(X[i-1] - VI[j])*np.random.normal(0,delta)
            if Xi > S :
                X.append(V0);
                spikes.append(spikingTime)
                spikeDistrib.append( spikingTime-spikeDistrib[-1])
            else:
                X.append(Xi);
                spikingTime +=1 ;
        Xlist.append(X);
        spikeDistribList.append(spikeDistrib);
            
    scatter("time",Xlist,VIname,"Euler scheme : Feller process ")    
    histogram(spikeDistribList,'Spike Distribution','time')
    return X

In [13]:
X = EulerFeller(0.001, 5000,0.5, 50, -65,[-70], 10,-45)

In [14]:
X = EulerFeller(0.001, 5000,0.5, 50, -65,[-140, -90,-80,-70], 10,-45)